In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v3 import *
from dnn_utils_v2 import sigmoid,sigmoid_backward,relu,relu_backward

%matplotlib inline
plt.rcParams['figure.figsize']=(5.0,4.0)  #set default size of plots
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [3]:
def sigmoid(Z):
    A=1/(1+np.exp(-Z))
    cache=Z
    
    return A,cache

In [4]:
def sigmoid_backward(dA,cache):
    
    Z=cache
    s=1/(1+np.exp(-Z))
    dZ=dA*s*(1-s)
    
    assert(dZ.shape==Z.shape)
    
    return dZ

In [5]:
#the relu function
def relu(Z):
    A=np.maximum(0,Z)
    
    assert(A.shape==Z.shape)
    
    cache=Z
    return A,cache

In [6]:
def relu_backward(dA,cache):
    z=cache
    dZ=np.array(dA,copy=True) #just converting dz to a correct object
    
    #when z<=0,you should set dz to 0 as well
    dZ[z<=0]=0
    
    assert(dZ.shape==Z.shape)
    
    return dZ
    

In [7]:
#initialization for 2-layer neural network
def initial_parameters(n_x,n_h,n_y):
    np.random.seed(1)
    
    W1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros((n_h,1))
    W2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros((n_y,1))
    
    assert(W1.shape==(n_h,n_x))
    assert(b1.shape==(n_h,1))
    assert(W2.shape==(n_y,n_h))
    assert(b2.shape==(n_y,1))
    
    parameters={
        "W1":W1,
        "b1":b1,
        "W2":W2,
        "b2":b2
    }
    
    return parameters

In [8]:
#initialization for L-layer neural network

def initialize_parameters_deep(layer_dims):
    
    np.random.seed(3)
    parameters={}
    L=len(layer_dims)  #神经网络中有多少层
    
    for l in range(1,L):
        
        parameters['W'+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b'+str(l)]=np.zeros((layer_dims[l],1))
        
        assert(parameters['W'+str(l)].shape==(layer_dims[l],layer_dims[l-1]))
        assert(parameters['b'+str(l)].shape==(layer_dims[l],1))
        
    return parameters

In [9]:
#线性正向传播

def linear_forward(A,W,b):
    
    Z=np.dot(W,A)+b
    
    assert(Z.shape==(W.shape[0],A.shape[1]))
    cache=(A,W,b)
    
    return Z,cache

In [10]:
#线性激活函数传播

def linear_activation_forward(A_prev,W,b,activation):
    
    if activation=="sigmoid":
        Z,linear_cache=linear_forward(A_prev,W,b)
        A,activation_cache=sigmoid(Z)
    
    elif activation=="relu":
        Z,linear_cache=linear_forward(A_prev,W,b)
        A,activation_cache=relu(Z)
        
    assert(A.shape==(W.shape[0],A_prev.shape[1]))
    cache=(linear_cache,activation_cache)
    
    return A,cache

In [11]:
#L模型传播

def L_model_forward(X,parameters):
    
    caches=[]
    A=X
    L=len(parameters)//2
    
    for l in range(1,L):
        A_prev=A
        A,cache=linear_activation_forward(A_prev,parameters['W'+str(l)],parameters['b'+str(l)],"relu")
        caches.append(cache)
    
    AL,cache=linear_activation_forward(A,parameters['W'+str(L)],parameters['b'+str(L)],"sigmoid")
    caches.append(cache)
    
    assert(AL.shape==(1,X.shape[1]))
    
    return AL,caches

In [12]:
#计算代价函数

def compute_cost(AL,Y):
    
    m=Y.shape[1]
    cost=-np.sum(np.multiply(np.log(AL),Y)+np.multply(np.log(1-AL),1-Y))/m
    cost=np.squeeze(cost)
    
    assert(cost.shape==())
    
    return cost

In [13]:
#线性反向传播

def linear_backward(dZ,cache):
    A_prev,W,b=cache
    m=A_prev.shape[1]
    
    dW=np.dot(dZ,A_prev.T)/m
    db=np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev=np.dot(W.T,dZ)
    
    assert(dA_prev.shape==A_prev.shape)
    assert(dW.shape==W.shape)
    assert(db.shape==b.shape)
    
    return dA_prev,dW,db    

In [16]:
#线性激活函数反向传播

def linear_activation_backward(dA,cache,activation):
    
    linear_cache,activation_cache=cache
    
    if activation=="relu":
        dZ=relu_backward(dA,activation_cache)
        dA_prev,dW,db=linear_backward(dZ,linear_cache)
        
    elif activation=="sigmoid":
        dZ=sigmoid_backward(dA,activation_cache)
        dA_prev,dW,db=linear_backward(dZ,linear_cache)
        
    return dA_prev,dW,db

In [17]:
#L层模型的向后传播

def L_model_backward(AL,Y,caches):
    
    grads={}
    L=len(caches)  #number of caches
    m=AL.shape[1]
    Y=Y.reshape[AL.shape]
    
    dAL=-(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    current_cache=caches[L-1]
    grads["dA"+str(L)],grads["dW"+str(L)],grads["db"+str(L)]=linear_activation_backward(dAL,current_cache,"sigmoid")
    
    for l in reversed(range(L-1)):
        current_cache=caches[l]
        dA_prev_temp,dW_temp,db_temp=linear_activation_backward(grads["dA"+str(l+2)],current_cache,"relu")
        grads["dA"+str(l+1)]=dA_prev_temp
        grads["dW"+str(l+1)]=dW_temp
        grads["db"+str(l+1)]=db_temp
    
    return grads

In [18]:
#更新参数

def update_parameters(parameters,grads,learning_rate):
    
    L=len(parameters)//2  #numbers of layers in the neural network
    
    for l in range(L):
        parameters["W"+str(l+1)]=parameters["W"+str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        parameters["b"+str(l+1)]=parameters["b"+str(l+1)]-learning_rate*grads["db"+str(l+1)]
        
    return parameters